In [1]:
import cv2
import numpy as np
import sys

# --------- Config (tune as needed) ----------
SHORT_SIDE_TARGET = 720
CANNY_LOWER, CANNY_UPPER = 100, 200
MORPH_CLOSE_W_RATIO = 0.015
MIN_AR, MAX_AR = 2.0, 6.0
MIN_AREA_FRAC, MAX_AREA_FRAC = 0.002, 0.2
RECTANGULARITY_MIN = 0.6
EDGE_DENSITY_MIN = 0.03
PLATE_W, PLATE_H = 240, 80

def wait(msg="Press any key (q/ESC to quit)"):
    k = cv2.waitKey(0) & 0xFF
    if k in (27, ord('q')):  # ESC or q
        cv2.destroyAllWindows()
        sys.exit(0)

def resize_keep_aspect(img, short_target):
    h, w = img.shape[:2]
    if min(h, w) == short_target:
        return img
    if h < w:
        new_h = short_target
        new_w = int(w * (short_target / h))
    else:
        new_w = short_target
        new_h = int(h * (short_target / w))
    return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

def enhance_gray(gray):
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(blur)
    return clahe

def connect_plate_bands(edges, width):
    se_w = max(3, int(width * MORPH_CLOSE_W_RATIO))
    kernel_h = cv2.getStructuringElement(cv2.MORPH_RECT, (se_w, 3))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel_h, iterations=1)
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel_h, iterations=1)
    return opened

def contour_rectangularity(cnt):
    area = cv2.contourArea(cnt)
    x,y,w,h = cv2.boundingRect(cnt)
    rect_area = w*h if w*h > 0 else 1
    return area / rect_area

def edge_density_in_rect(edges, rect):
    x,y,w,h = rect
    roi = edges[max(0,y):y+h, max(0,x):x+w]
    if roi.size == 0:
        return 0.0
    return float(np.count_nonzero(roi)) / float(roi.size)

def approx_plate_quad(cnt):
    peri = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
    if len(approx) == 4:
        return approx.reshape(4,2)
    return None

def order_quad_points(pts):
    rect = np.zeros((4,2), dtype="float32")
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    rect[0] = pts[np.argmin(s)]   # tl
    rect[2] = pts[np.argmax(s)]   # br
    rect[1] = pts[np.argmin(diff)]# tr
    rect[3] = pts[np.argmax(diff)]# bl
    return rect

def warp_perspective(image, quad_pts, out_w=PLATE_W, out_h=PLATE_H):
    rect = order_quad_points(quad_pts.astype("float32"))
    dst = np.array([[0,0],[out_w-1,0],[out_w-1,out_h-1],[0,out_h-1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (out_w, out_h), flags=cv2.INTER_CUBIC)
    return warped

def main(img_path):
    bgr = cv2.imread(img_path)
    if bgr is None:
        print("Failed to read image")
        return

    # 1) Original
    cv2.imshow("1 - Original (press key)", bgr); wait()

    # 2) Resize normalize
    img = resize_keep_aspect(bgr, SHORT_SIDE_TARGET)
    cv2.imshow("2 - Resized", img); wait()

    # 3) Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imshow("3 - Grayscale", gray); wait()

    # 4) Enhance (denoise + CLAHE)
    gray_enh = enhance_gray(gray)
    cv2.imshow("4 - Enhanced (Blur+CLAHE)", gray_enh); wait()

    # 5) Edges (Canny)
    edges = cv2.Canny(gray_enh, CANNY_LOWER, CANNY_UPPER)
    cv2.imshow("5 - Canny Edges", edges); wait()

    # 6) Morphology to connect plate band
    bands = connect_plate_bands(edges, img.shape[1])
    cv2.imshow("6 - Morph Connected Bands", bands); wait()

    # 7) Contours and candidate visualization
    cnts, _ = cv2.findContours(bands, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    vis_cnt = img.copy()
    cv2.drawContours(vis_cnt, cnts, -1, (0,255,0), 2)
    cv2.imshow("7 - All Contours", vis_cnt); wait()

    h, w = img.shape[:2]
    area_img = h * w
    candidates = []
    for c in cnts:
        x,y,ww,hh = cv2.boundingRect(c)
        ar = ww / float(hh) if hh > 0 else 999.0
        area_c = ww * hh
        if not (MIN_AREA_FRAC*area_img <= area_c <= MAX_AREA_FRAC*area_img):
            continue
        if not (MIN_AR <= ar <= MAX_AR):
            continue
        rectness = contour_rectangularity(c)
        if rectness < RECTANGULARITY_MIN:
            continue
        ed = edge_density_in_rect(edges, (x,y,ww,hh))
        if ed < EDGE_DENSITY_MIN:
            continue
        quad = approx_plate_quad(c)
        if quad is None:
            quad = np.array([[x,y],[x+ww,y],[x+ww,y+hh],[x,y+hh]], dtype=np.int32)
        candidates.append({"quad": quad, "bbox": (x,y,ww,hh), "score": rectness*ed})

    # 8) Show candidate boxes
    vis_cand = img.copy()
    for cand in candidates:
        x,y,ww,hh = cand["bbox"]
        cv2.rectangle(vis_cand, (x,y), (x+ww,y+hh), (0,0,255), 2)
    cv2.imshow("8 - Filtered Candidates", vis_cand); wait()

    # 9) Warp best candidate (if any)
    if candidates:
        candidates.sort(key=lambda d: d["score"], reverse=True)
        best = candidates[0]
        warped = warp_perspective(img, best["quad"], PLATE_W, PLATE_H)
        cv2.imshow("9 - Rectified Plate (Best)", warped); wait()
    else:
        blank = np.zeros((PLATE_H, PLATE_W, 3), dtype=np.uint8)
        cv2.putText(blank, "No candidate found", (10, PLATE_H//2), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
        cv2.imshow("9 - Rectified Plate (Best)", blank); wait()

    cv2.destroyAllWindows()

if __name__ == "__main__":
    
    main("E:\Indian_Number_Plates\Sample_Images\Success\Datacluster_number_plates (66).jpg")

    if len(sys.argv) < 2:
        print("Usage: python detect_plate_show.py /path/to/image.jpg")
        sys.exit(1)
    


<>:161: SyntaxWarning: invalid escape sequence '\I'
<>:161: SyntaxWarning: invalid escape sequence '\I'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15748\790099070.py:161: SyntaxWarning: invalid escape sequence '\I'
  main("E:\Indian_Number_Plates\Sample_Images\Success\Datacluster_number_plates (66).jpg")


SystemExit: 0

D:\Newfolder\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import os, glob, math
import cv2
import numpy as np

# --------- Config ----------
INPUT_DIR = "dataset_images"          # folder with input images
OUTPUT_DIR = "detected_plates"        # folder to save cropped plates
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Detection params (tune per camera)
SHORT_SIDE_TARGET = 720               # resize shorter side to stabilize thresholds
CANNY_LOWER, CANNY_UPPER = 100, 200   # Canny thresholds
MORPH_CLOSE_W_RATIO = 0.015           # horizontal SE width as fraction of image width
MIN_AREA_FRAC = 0.002                 # min contour area as fraction of image area
MAX_AREA_FRAC = 0.2                   # max contour area as fraction of image area
MIN_AR, MAX_AR = 2.0, 6.0             # plate aspect ratio range (w/h)
RECTANGULARITY_MIN = 0.6              # contour area / bounding-rect area
EDGE_DENSITY_MIN = 0.03               # edge pixels / candidate area
PLATE_W, PLATE_H = 240, 80            # canonical rectified size

def resize_keep_aspect(img, short_target):
    h, w = img.shape[:2]
    if min(h, w) == short_target:
        return img
    if h < w:
        new_h = short_target
        new_w = int(w * (short_target / h))
    else:
        new_w = short_target
        new_h = int(h * (short_target / w))
    return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

def enhance_gray(gray):
    # mild denoise + local contrast
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(blur)
    return clahe

def edge_map(gray_enhanced):
    # Canny edge detection
    edges = cv2.Canny(gray_enhanced, CANNY_LOWER, CANNY_UPPER)
    return edges

def connect_plate_bands(edges, width):
    # Morphological closing then opening with horizontal SE to connect character strokes
    se_w = max(3, int(width * MORPH_CLOSE_W_RATIO))
    kernel_h = cv2.getStructuringElement(cv2.MORPH_RECT, (se_w, 3))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel_h, iterations=1)
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel_h, iterations=1)
    return opened

def contour_rectangularity(cnt):
    area = cv2.contourArea(cnt)
    x,y,w,h = cv2.boundingRect(cnt)
    rect_area = w*h if w*h > 0 else 1
    return area / rect_area

def edge_density_in_rect(edges, rect):
    x,y,w,h = rect
    roi = edges[max(0,y):y+h, max(0,x):x+w]
    if roi.size == 0:
        return 0.0
    return float(np.count_nonzero(roi)) / float(roi.size)

def approx_plate_quad(cnt):
    # Approximate to polygon and try to get a 4-point contour
    peri = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
    if len(approx) == 4:
        return approx.reshape(4,2)
    return None

def order_quad_points(pts):
    # Order points as [tl, tr, br, bl]
    rect = np.zeros((4,2), dtype="float32")
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    rect[0] = pts[np.argmin(s)]  # tl
    rect[2] = pts[np.argmax(s)]  # br
    rect[1] = pts[np.argmin(diff)] # tr
    rect[3] = pts[np.argmax(diff)] # bl
    return rect

def warp_perspective(image, quad_pts, out_w=PLATE_W, out_h=PLATE_H):
    rect = order_quad_points(quad_pts.astype("float32"))
    dst = np.array([[0,0],[out_w-1,0],[out_w-1,out_h-1],[0,out_h-1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (out_w, out_h), flags=cv2.INTER_CUBIC)
    return warped

def find_plate_candidates(image_bgr):
    img = resize_keep_aspect(image_bgr, SHORT_SIDE_TARGET)
    h, w = img.shape[:2]
    area_img = h * w

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_enh = enhance_gray(gray)
    edges = edge_map(gray_enh)
    bands = connect_plate_bands(edges, w)

    # Find external contours on band image
    cnts, _ = cv2.findContours(bands, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    candidates = []
    for c in cnts:
        x,y,ww,hh = cv2.boundingRect(c)
        ar = ww / float(hh) if hh > 0 else 999.0
        area_c = ww * hh
        if area_c < MIN_AREA_FRAC*area_img or area_c > MAX_AREA_FRAC*area_img:
            continue
        if ar < MIN_AR or ar > MAX_AR:
            continue
        rectness = contour_rectangularity(c)
        if rectness < RECTANGULARITY_MIN:
            continue
        ed = edge_density_in_rect(edges, (x,y,ww,hh))
        if ed < EDGE_DENSITY_MIN:
            continue

        # Prefer quadrilateral if available for better warp
        quad = approx_plate_quad(c)
        if quad is None:
            quad = np.array([[x,y],[x+ww,y],[x+ww,y+hh],[x,y+hh]], dtype=np.int32)

        candidates.append({
            "quad": quad,
            "bbox": (x,y,ww,hh),
            "score": rectness * ed
        })

    # Sort best-first
    candidates.sort(key=lambda d: d["score"], reverse=True)
    return img, candidates

def process_dataset():
    image_paths = []
    for ext in ("*.jpg","*.jpeg","*.png","*.bmp"):
        image_paths.extend(glob.glob(os.path.join(INPUT_DIR, ext)))
    image_paths.sort()

    results = []
    for idx, p in enumerate(image_paths):
        bgr = cv2.imread(p)
        if bgr is None:
            print(f"[WARN] Failed to read {p}")
            continue
        img_resized, cands = find_plate_candidates(bgr)
        basename = os.path.splitext(os.path.basename(p))[0]
        if not cands:
            results.append((p, 0))
            continue

        # Save top-K candidates; start with top-1 for simplicity
        K = min(3, len(cands))
        for k in range(K):
            quad = cands[k]["quad"]
            plate = warp_perspective(img_resized, quad, PLATE_W, PLATE_H)
            out_path = os.path.join(OUTPUT_DIR, f"{basename}_plate{k+1}.png")
            cv2.imwrite(out_path, plate)
        results.append((p, K))

    # Simple report
    total = len(image_paths)
    with_cand = sum(1 for _, k in results if k > 0)
    print(f"Processed {total} images, found candidates in {with_cand} images.")

if __name__ == "__main__":
    process_dataset()


In [24]:
import cv2
import numpy as np
import sys

# ----- Config (tune as needed) -----
SHORT_SIDE_TARGET = 720
CANNY_LOWER, CANNY_UPPER = 100, 200
MORPH_CLOSE_W_RATIO = 0.015
MIN_AR, MAX_AR = 2.0, 6.0
MIN_AREA_FRAC, MAX_AREA_FRAC = 0.002, 0.2
RECTANGULARITY_MIN = 0.6
EDGE_DENSITY_MIN = 0.03
PLATE_W, PLATE_H = 240, 80

IMG_PATH = r"E:\Indian_Number_Plates\Sample_Images\Datacluster_number_plates (66).jpg"

def wait_or_quit():
    k = cv2.waitKey(0) & 0xFF
    if k in (27, ord('q')):  # ESC or q
        cv2.destroyAllWindows()
        raise SystemExit

def resize_keep_aspect(img, short_target):
    h, w = img.shape[:2]
    if h < w:
        new_h = short_target
        new_w = int(w * (short_target / h))
    else:
        new_w = short_target
        new_h = int(h * (short_target / w))
    return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

def enhance_gray(gray):
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(blur)
    return clahe

def connect_plate_bands(edges, width):
    se_w = max(3, int(width * MORPH_CLOSE_W_RATIO))
    kernel_h = cv2.getStructuringElement(cv2.MORPH_RECT, (se_w, 3))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel_h, iterations=1)
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel_h, iterations=1)
    return opened

def contour_rectangularity(cnt):
    area = cv2.contourArea(cnt)
    x,y,w,h = cv2.boundingRect(cnt)
    rect_area = max(1, w*h)
    return area / rect_area

def edge_density_in_rect(edges, rect):
    x,y,w,h = rect
    roi = edges[max(0,y):y+h, max(0,x):x+w]
    if roi.size == 0:
        return 0.0
    return float(np.count_nonzero(roi)) / float(roi.size)

def approx_plate_quad(cnt):
    peri = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
    if len(approx) == 4:
        return approx.reshape(4,2)
    return None

def order_quad_points(pts):
    rect = np.zeros((4,2), dtype="float32")
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    rect[0] = pts[np.argmin(s)]   # tl
    rect[2] = pts[np.argmax(s)]   # br
    rect[1] = pts[np.argmin(diff)]# tr
    rect[3] = pts[np.argmax(diff)]# bl
    return rect

def warp_perspective(image, quad_pts, out_w=PLATE_W, out_h=PLATE_H):
    rect = order_quad_points(quad_pts.astype("float32"))
    dst = np.array([[0,0],[out_w-1,0],[out_w-1,out_h-1],[0,out_h-1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (out_w, out_h), flags=cv2.INTER_CUBIC)
    return warped
